# 演習２：用例ベース雑談システム

概要
* 用例ベース（検索ベース）の雑談対話システムを作成
* 入力発話とシステム応答の候補との類似度計算には、SentenceBERTならびにコサイン類似度を使用

目的
* 「こう聞かれたら、こう答える」という一問一答型の音声対話システムの実装を体験する

下記のような用例ベース対話システムを実装します。
あらかじめ想定されるユーザ発話とそれに対応するシステム応答のペアを複数用意しておき、ユーザ発話が入力されたら最も近い想定ユーザ発話を検索します。
そして、それに対応するシステム応答を出力します。
類似度の計算には、SentenceBERTとコサイン距離を用います。

<img src="./img/example.png" style="width: 600px;"/>

## 必要なライブラリのインストール

はじめに、必要となるライブラリをインストールします。ここではpipを使ってインストールします。

In [1]:
# SentenceBERT 
! pip install sentence-transformers
! pip install fugashi
! pip install ipadic

# 音声認識
! pip install pyaudio
! pip install SpeechRecognition

# 音声合成
! pip install gTTS
! pip install pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 734.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.3 MB/s eta 0:00:00
  Using cached torch-2.3.0-cp39-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
  Using cached pillow-10.3.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.8 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached MarkupSafe-2.1.5-cp39-cp39-macosx_10_9_universal2.whl.metadata (3.0 kB)
  Using cached mpmath-1.3

## Step 1: 用例データの用意

想定されるユーザ発話とそれに対応するシステム応答のペアデータ（用例データ）を読み込みます。
データは data/example-base-data.csvに格納されており、各行が１つのペアデータ、１列目が想定ユーザ発話、２列目がシステム応答です。

In [2]:
# 用例データを読み込む
pair_data = []
filename = './data/example-base-data.csv'
print('Load from %s' % filename)
with open(filename, 'r', encoding='utf8') as f:
    lines = f.readlines()
    for line in lines:
        u1 = line.split(',')[0].strip()
        u2 = line.split(',')[1].strip()
        pair_data.append([u1, u2])
        
        print('%s -> %s' % (u1, u2))

Load from ./data/example-base-data.csv
こんにちは -> こんにちは
趣味は何ですか -> 趣味はスポーツ観戦です
好きな食べ物は何ですか -> りんごです
一番安い商品は何ですか -> 一番安いのはもやしです
最近食べた料理は何ですか -> 最近食べたのはラーメンです
出身はどこですか -> 出身は京都です
印象に残っている旅行はなんですか -> 印象に残っているのはヨーロッパ旅行です
おはよう -> おはようございます
こんばんは -> こんばんは
さようなら -> さようなら
好きな芸能人は誰ですか -> 好きな芸能人はタモリです
好きなスポーツは何ですか -> 好きなスポーツはサッカーです
好きな動物は何ですか -> 好きな動物は犬です
休日は何をされていますか -> 休日は主に散歩しています
仕事は何をしていますか -> 仕事は受付係をしています
最近はまっていることは何ですか -> 最近は映画鑑賞にはまっています
好きな映画は何ですか -> 好きな映画はスターウォーズです
好きなゲームは何ですか -> 好きなゲームはポケモンです
好きなポケモンは何ですか -> 好きなポケモンはピカチュウです
好きな本は何ですか -> 好きな本は純粋理性批判です
好きな小説は何ですか -> 好きな小説は指輪物語です
好きな漫画は何ですか -> 好きな漫画はドラえもんです
好きなアニメは何ですか -> 好きなアニメはドラゴンボールです
おすすめのお店はどこですか -> おすすめのお店はサイゼリアです
得意な料理は何ですか -> チャーハンが得意料理です
好きな教科は何ですか -> 好きな教科は数学です
おすすめの観光地はどこですか -> おすすめは清水寺です
おすすめのお土産は何ですか -> おすすめは八つ橋です
好きなボードゲームは何ですか -> カタンです
好きな数字は何ですか -> 好きな数字は1です
印象に残っている映画は何ですか -> 印象に残っているのはシャイニングです
好きな季節は何ですか -> 好きな季節は夏です
嫌いな食べ物は何ですか -> 嫌いな食べ物はキウイです
嫌いな動物は何ですか -> 嫌いな動物はカラスです
何歳ですか -> 20歳です
健康にはどのように気を付けていますか -> 毎日運動するようにしています

## Step 2: SentenceBERTによる文ベクトル変換

読み込んだ用例のデータのうち、想定ユーザ発話の各発話文をSentenceBERTを用いてベクトル化します。

SentenceBERTのモデルには、今回は下記のものを用います。
https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2

はじめに、上記のサイトを参考に、SentenceBERTを利用するためのクラスを作成します。

In [3]:
# SentenceBERTで使用
from transformers import BertJapaneseTokenizer, BertModel
import torch

# Sentence-BERTの日本語版モデルを操作するためのクラス
class SentenceBertJapanese:
    
    # コンストラクタ
    # model_name_or_path: Sentence-BERTのモデル名またはパス
    # device: 使用するデバイス（CPU or GPU）の指定。デフォルトでは利用可能な場合はGPUを使用。今回の演習ではCPUを想定する。
    def __init__(self, model_name_or_path, device=None):
        
        # トークナイザの初期化
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        
        # モデルの初期化
        self.model = BertModel.from_pretrained(model_name_or_path)
        
        # 推論モードにモデルを設定
        self.model.eval()

        # 使用するデバイスの設定
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        
        # モデルを指定したデバイスに移動
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):

        # モデルの出力からトークンの埋め込みを取得
        token_embeddings = model_output[0]
        
        # attention_maskをトークン埋め込みの次元に展開
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        
        # トークンの埋め込みを平均プーリングして文の埋め込みを取得
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # 文のリストをベクトルに変換するメソッド
    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        
        for batch_idx in iterator:
            
            # 文をバッチ処理するための部分集合を取得
            batch = sentences[batch_idx:batch_idx + batch_size]
            
            # 文をトークン化してモデル入力用にエンコード
            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            
            # モデルを使用してエンコードされた入力から出力を取得
            model_output = self.model(**encoded_input)
            
            # 平均プーリングを使用して文の埋め込みを取得
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')
            
            # 全ての文の埋め込みをリストに追加
            all_embeddings.extend(sentence_embeddings)

        # 最終的な文の埋め込みのテンソルを返す
        return torch.stack(all_embeddings)


/Users/nagairyousuke/Library/Mobile Documents/com~apple~CloudDocs/lesson/分野別演習/Hands-on/Voice/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nagairyousuke/Library/Mobile Documents/com~apple~CloudDocs/lesson/分野別演習/Hands-on/Voice/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceBERTを試してみます。

In [4]:
MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"
model = SentenceBertJapanese(MODEL_NAME)

sentences = ["京都大学へようこそ", "京都でおいしいごはんを食べた"]
sentence_embeddings = model.encode(sentences, batch_size=8)

print("Sentence embeddings 1:", sentence_embeddings[0])
print("len(Sentence embeddings 1):", len(sentence_embeddings[0]))
      
print("Sentence embeddings 2:", sentence_embeddings[1])
print("len(Sentence embeddings 2):", len(sentence_embeddings[1]))

Sentence embeddings 1: tensor([ 5.9404e-01,  1.5339e-01, -1.9529e-01, -7.2835e-01, -5.1741e-02,
         6.9446e-01,  8.5812e-02,  1.5175e-01,  3.1300e-01, -5.1158e-01,
        -6.6770e-01,  1.5187e-02,  1.5812e+00, -2.3450e-01, -1.5493e-01,
         5.5953e-02,  8.3474e-01, -9.0972e-01,  7.4528e-01,  4.2427e-01,
        -9.5427e-01, -1.1214e+00,  4.1615e-01,  1.6549e+00,  1.7530e-01,
         2.6579e-01,  4.2268e-01,  2.1663e-01,  1.7317e-02, -8.3101e-01,
         1.5769e-01,  3.4213e-01,  9.7243e-01, -5.5689e-02,  1.3607e-01,
        -1.0779e+00,  1.7090e+00,  8.7365e-01,  6.4724e-01, -1.2737e+00,
         1.5321e-01,  1.7465e-01, -1.1294e+00, -2.6936e-01, -3.5041e-01,
        -4.6568e-01, -4.8545e-01, -7.9053e-02, -3.1532e-02, -1.1148e-01,
         5.4962e-02,  7.0084e-01,  3.1852e-01, -2.1838e-01, -3.9531e-01,
         2.8030e-01, -3.9804e-01, -1.8922e-01,  2.6280e-01, -3.1478e-01,
        -7.5026e-01, -1.4440e-01,  2.8241e-01, -1.3679e-01, -1.5675e-02,
        -3.6194e-01, -2.0804

上記のmodelを用いて、各用例データを文ベクトル化します。

In [5]:
# 用例データをSentence-BERTでベクトル化
pair_data_vec = []
for d in pair_data:
    u1 = model.encode([d[0]])[0]
    u2 = d[1]
    pair_data_vec.append([u1, u2])
    
print(pair_data_vec[0][0])
print(pair_data_vec[0][1])

tensor([-3.5213e-01, -8.2440e-01,  1.6509e+00, -6.6546e-01, -2.9563e-01,
        -3.7789e-01, -7.6682e-01, -1.8869e+00,  4.0268e-01, -7.9262e-01,
        -3.7024e-01, -4.2161e-01,  4.2909e-01,  8.8229e-01,  4.3958e-01,
        -1.0033e-02,  5.0678e-01, -1.3921e-01, -3.9400e-01,  7.8616e-01,
         7.0085e-01,  6.1590e-01, -2.1692e-01, -9.3460e-02, -8.9282e-01,
         1.4738e-01, -1.1537e+00,  5.5465e-02, -4.7788e-01,  3.6574e-01,
         1.8132e+00, -4.3117e-01, -3.5694e-01,  2.0684e-01,  7.4967e-01,
        -1.1176e+00, -4.5692e-01,  9.8977e-01,  7.5425e-01, -1.3753e-01,
         2.6688e-01,  1.2967e+00, -4.9476e-01, -1.1798e-01, -4.2036e-01,
        -1.8607e-02,  6.0879e-01,  4.9399e-01,  4.9975e-01, -1.1617e-01,
         8.9487e-01, -3.2889e-01, -7.4938e-01,  4.9973e-01,  6.1867e-01,
         1.0315e+00,  8.1616e-01, -6.4796e-01,  1.9505e-01,  3.2211e-01,
         2.4184e-01, -1.6260e-01, -4.7261e-02,  3.8767e-01,  4.7321e-01,
        -1.3837e-01, -3.5998e-01, -2.7380e-01,  4.0

## Step 3: 類似度計算

次に、類似度を計算する関数を用意します。
ここでは、入力ユーザ発話と、用例データを受け取り、入力ユーザ発話に最も類似するシステム応答を返します。

In [6]:
# コサイン類似度を計算する際にnumpyを使用
import numpy as np

# 類似度計算
# input_sentence_vec: ベクトル化された入力ユーザ発話
# pair_data_vec: ベクトル化された用例データ
def matching(input_sentence_vec: np.array, pair_data_vec: list):
    
    # コサイン類似度が最も高いものを採用
    cos_dist_max = 0.
    response = None
    
    # 用例毎に処理
    for pair_each in pair_data_vec:
        
        # コサイン類似度を計算
        v1 = input_sentence_vec
        v2 = pair_each[0]
        cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        
        # 最大値を更新
        if cos_dist_max < cos_sim:
            cos_dist_max = cos_sim
            response = pair_each[1]
    
    return response, cos_dist_max
        

## Step 3: テスト

では、用例ベース対話システムをテストしてみましょう。

In [7]:
# テスト

# 入力発話　その１
input_sentence = '趣味は何ですか'
input_sentence_vec = model.encode([input_sentence])[0]

response, cos_dist_max = matching(input_sentence_vec, pair_data_vec)

print('入力：%s' % input_sentence)
print('応答：%s' % response)
print('類似度：%.3f' % cos_dist_max)
print()

# 入力発話　その２
input_sentence = '最近面白かったものは何ですか'
input_sentence_vec = model.encode([input_sentence])[0]

response, cos_dist_max = matching(input_sentence_vec, pair_data_vec)

print('入力：%s' % input_sentence)
print('応答：%s' % response)
print('類似度：%.3f' % cos_dist_max)
print()

入力：趣味は何ですか
応答：趣味はスポーツ観戦です
類似度：1.000

入力：最近面白かったものは何ですか
応答：最近は毎日を充実しています
類似度：0.774



### Step 4: 音声対話システムとしての統合

最後に、演習1で実装した方法を用いて、音声対話システムとして動作するように上記の機能を統合します。

In [8]:
# ライブラリのインポート
import speech_recognition as sr
from gtts import gTTS
import pygame

# 音声認識を関数化
def get_asr():
    
    r = sr.Recognizer()
    r.pause_threshold = 0.5
    
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source) # 背景雑音へ適応する（１秒間）
        print("どうぞ話してください >> ")
        audio = r.listen(source)
    
    try:
        result = r.recognize_google(audio, language="ja-JP")
    except sr.UnknownValueError:
        result = ""
    except sr.RequestError as e:
        result = ""
    
    return result

# 音声合成を関数化
def play_tts(text):
    
    speech = gTTS(text=text, lang="ja")

    try:
        speech.save("./data/test.mp3")
    except Exception as e:
        print('ファイル保存エラー')
    
    pygame.mixer.init()
    pygame.mixer.music.load("./data/test.mp3")
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

    pygame.mixer.music.stop()
    pygame.mixer.quit()

pygame 2.5.2 (SDL 2.28.3, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:
# 対話が終了状態に移るまで対話を続ける
while True:
    
    # 音声入力＆音声認識
    result_asr_utterance = get_asr()
    print("ユーザ： " + result_asr_utterance)
    
    # 「終了」がユーザ発話に含まれていれば対話を終了
    if "終了" in result_asr_utterance:
        break

    # 用例を検索
    input_sentence_vec = model.encode([result_asr_utterance])[0]
    system_utterance, cos_dist_max = matching(input_sentence_vec, pair_data_vec)

    print("システム： " + system_utterance)
    play_tts(system_utterance)
    
    print()

# 対話終了
print("対話終了")

どうぞ話してください >> 
ユーザ： 
システム： おはようございます

どうぞ話してください >> 
ユーザ： おはようございます
システム： おはようございます

どうぞ話してください >> 
ユーザ： あなたの好きな食べ物なんですか
システム： りんごです

どうぞ話してください >> 
ユーザ： なぜ りんごが好きなのですか
システム： りんごです

どうぞ話してください >> 
ユーザ： 中学時代の思い出は何ですか
システム： みんなで海外旅行に行ったのが思い出です

どうぞ話してください >> 
ユーザ： 最近僕は引っ越したのですが あなたは一人暮らしですか
システム： 今は一人で暮らしています

どうぞ話してください >> 
ユーザ： へーそうなんですね
システム： よろしくお願いします

どうぞ話してください >> 


ConnectionResetError: [Errno 54] Connection reset by peer